## Exercise 9.2: Best Model Selection and Hyperparameter Tuning

In [1]:
# First I will import some needed libraries
import pandas as pd
from importlib import reload
import sys
import numpy as np
from imp import reload
import nltk
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec 
%matplotlib inline
import seaborn as sns 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
import re
nltk.download('wordnet')
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
warnings.filterwarnings('ignore')
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

[nltk_data] Downloading package wordnet to /Users/Robyn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Step 1: Import the dataset and ensure that it loaded properly.

In [296]:
# I will use pandas to pull the data to create a data frame to work from
Loan_Train = pd.read_csv('Loan_Train.csv')
Loan_Train

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


## Step 2: Prepare the data for modeling by performing the following steps:

## Drop the column “Load_ID.”

In [297]:
# I will first drop my Loan_ID variable with the use of drop
Loan_Train.drop(columns =["Loan_ID"], inplace = True)
Loan_Train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


## Drop any rows with missing data.

In [298]:
# Next I will check for any missing data
Loan_Train.isnull().sum()/len(Loan_Train)

Gender               0.021173
Married              0.004886
Dependents           0.024430
Education            0.000000
Self_Employed        0.052117
ApplicantIncome      0.000000
CoapplicantIncome    0.000000
LoanAmount           0.035831
Loan_Amount_Term     0.022801
Credit_History       0.081433
Property_Area        0.000000
Loan_Status          0.000000
dtype: float64

In [299]:
# I will now drop all variables with missing data with the use of drop
Loan_Train.dropna(subset = ["Gender", "Married", "Dependents", "Self_Employed", "LoanAmount", "Loan_Amount_Term", "Credit_History"], inplace=True)
Loan_Train

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [300]:
# I will check and make sure there is no missing data
Loan_Train.isnull().sum()/len(Loan_Train)

Gender               0.0
Married              0.0
Dependents           0.0
Education            0.0
Self_Employed        0.0
ApplicantIncome      0.0
CoapplicantIncome    0.0
LoanAmount           0.0
Loan_Amount_Term     0.0
Credit_History       0.0
Property_Area        0.0
Loan_Status          0.0
dtype: float64

## Convert the categorical features into dummy variables

In [301]:
# First I will check and see the difference between the numerical features and 
# the categorical features
categorical_features = Loan_Train.select_dtypes(include=['object']).columns
print(categorical_features)
numerical_features = Loan_Train.select_dtypes(exclude = ["object"]).columns
print(numerical_features)

print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
feat_num = Loan_Train[numerical_features]
feat_cat = Loan_Train[categorical_features]

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Property_Area', 'Loan_Status'],
      dtype='object')
Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History'],
      dtype='object')
Numerical features : 5
Categorical features : 7


In [302]:
# Next I will view all categorical columns in the dataframe
feat_cat.head(10)

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,Rural,N
2,Male,Yes,0,Graduate,Yes,Urban,Y
3,Male,Yes,0,Not Graduate,No,Urban,Y
4,Male,No,0,Graduate,No,Urban,Y
5,Male,Yes,2,Graduate,Yes,Urban,Y
6,Male,Yes,0,Not Graduate,No,Urban,Y
7,Male,Yes,3+,Graduate,No,Semiurban,N
8,Male,Yes,2,Graduate,No,Urban,Y
9,Male,Yes,1,Graduate,No,Semiurban,N
10,Male,Yes,2,Graduate,No,Urban,Y


In [253]:
# I will now drop my original categorical variables with the use of drop
Loan_Train_Data.drop(columns =["Education", "Property_Area"], inplace = True)
Loan_Train_Data.head()

,Gender,Married,Dependents,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Graduate,Not Graduate,Rural,Semiurban,Urban,N,Y
1,Male,Yes,1,No,4583,1508.0,128.0,360.0,1.0,N,1,0,1,0,0,1,0
2,Male,Yes,0,Yes,3000,0.0,66.0,360.0,1.0,Y,1,0,0,0,1,0,1
3,Male,Yes,0,No,2583,2358.0,120.0,360.0,1.0,Y,0,1,0,0,1,0,1
4,Male,No,0,No,6000,0.0,141.0,360.0,1.0,Y,1,0,0,0,1,0,1
5,Male,Yes,2,Yes,5417,4196.0,267.0,360.0,1.0,Y,1,0,0,0,1,0,1


In [303]:
dummies = pd.get_dummies(Loan_Train[['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Property_Area', 'Loan_Status']], drop_first=True)
Loan_Data_Train = pd.concat([Loan_Train.drop(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Property_Area', 'Loan_Status'],axis=1), dummies],axis=1)
Loan_Data_Train

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
1,4583,1508.0,128.0,360.0,1.0,1,1,1,0,0,0,0,0,0,0
2,3000,0.0,66.0,360.0,1.0,1,1,0,0,0,0,1,0,1,1
3,2583,2358.0,120.0,360.0,1.0,1,1,0,0,0,1,0,0,1,1
4,6000,0.0,141.0,360.0,1.0,1,0,0,0,0,0,0,0,1,1
5,5417,4196.0,267.0,360.0,1.0,1,1,0,1,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,2900,0.0,71.0,360.0,1.0,0,0,0,0,0,0,0,0,0,1
610,4106,0.0,40.0,180.0,1.0,1,1,0,0,1,0,0,0,0,1
611,8072,240.0,253.0,360.0,1.0,1,1,1,0,0,0,0,0,1,1
612,7583,0.0,187.0,360.0,1.0,1,1,0,1,0,0,0,0,1,1


## Step 3: Split the data into a training and test set, where the “Loan_Status” column is the target.

In [304]:
# Printing all the column names for our reference
Loan_Data_Train.columns

Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Gender_Male', 'Married_Yes',
       'Dependents_1', 'Dependents_2', 'Dependents_3+',
       'Education_Not Graduate', 'Self_Employed_Yes',
       'Property_Area_Semiurban', 'Property_Area_Urban', 'Loan_Status_Y'],
      dtype='object')

In [305]:
# First I will separate the Target and Predictor Variables
Target = (['Loan_Status_Y'])
Predict= ['Gender_Male', 'Married_Yes',
       'Dependents_1', 'Dependents_2', 'Dependents_3+',
       'Education_Not Graduate', 'Self_Employed_Yes',
       'Property_Area_Semiurban', 'Property_Area_Urban']

X=Loan_Data_Train[Predict].values
Y=Loan_Data_Train[Target].values

# Then I will split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [306]:
# view the shape of my X
X.shape

(480, 9)

In [307]:
# view the shape of my Y
Y.shape

(480, 1)

## Step 4: Create a pipeline with a min-max scaler and a KNN classifier (see section 15.3 in the Machine Learning with Python Cookbook).

In [308]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion

# first I will create a min-max scaler 
PredictorScaler=MinMaxScaler()
# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit_transform(X_train)
# Next I will create a KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
# Last I will create a pipeline
pipe = Pipeline([("PredictorScalerFit", PredictorScaler), ("knn", knn)])

## Step 5: Fit a default KNN classifier to the data with this pipeline. Report the model accuracy on the test set. Note: Fitting a pipeline model works just like fitting a regular model.

In [309]:
# Next I will fit the classifier to the data
knn.fit(X_train,y_train)

KNeighborsClassifier(n_jobs=-1)

In [310]:
#Last I will check the accuracy of the model on the test data
knn.score(X_test, y_test)

0.6458333333333334

## Step 6: Create a search space for your KNN classifier where your “n_neighbors” parameter varies from 1 to 10. (see section 15.3 in the Machine Learning with Python Cookbook).

In [315]:
# Next I will create my search space for knn
search_space = [{"knn__n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}]

## Step 7: Fit a grid search with your pipeline, search space, and 5-fold cross-validation to find the best value for the “n_neighbors” parameter.

In [318]:
from sklearn.model_selection import GridSearchCV
# Next I will Fit the grid search for the n_neighbors
classifier = GridSearchCV(pipe, search_space, cv=5, verbose=0).fit(PredictorScalerFit, y_train)

## Step 8: Find the accuracy of the grid search best model on the test set. Note: It is possible that this will not be an improvement over the default model, but likely it will be.

In [319]:
# I will pull the k that produces the best model with the use of best_estimator_ and .get_params
classifier.best_estimator_.get_params()["knn__n_neighbors"]

5

In [367]:
# I will now pull the accuracy with use of score
classifier.score(X_test, y_test)

0.6458333333333334

## Step 9: Now, repeat steps 6 and 7 with the same pipeline, but expand your search space to include logistic regression and random forest models with the hyperparameter values in section 12.3 of the Machine Learning with Python Cookbook.

### Step 9.6: Create a search space for your KNN classifier where your “n_neighbors” parameter varies from 1 to 10. (see section 15.3 in the Machine Learning with Python Cookbook).

In [357]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Create a new pipeline that includes RandomForestClassifier

pipeline2 = Pipeline([('minmax', PredictorScaler), ('classifier', RandomForestClassifier())])

# I will first create my search space with logistic regression and random forest models
search_space2 = [{"classifier": [LogisticRegression()], "classifier__penalty": ['l1', 'l2'], 
                  "classifier__C": np.logspace(0, 4, 10)},
                 {"classifier": [RandomForestClassifier()],
                  "classifier__n_estimators": [10, 100, 1000], 
                  "classifier__max_features": [1, 2, 3]}]

In [358]:
gridsearch = GridSearchCV(pipeline2, search_space2, cv=5, verbose=True, n_jobs=-1)

### Step 9.7: Fit a grid search with your pipeline, search space, and 5-fold cross-validation to find the best value for the “n_neighbors” parameter.

In [359]:
bestmodel = gridsearch.fit(PredictorScalerFit, y_train)

Fitting 5 folds for each of 29 candidates, totalling 145 fits


## Step 10: What are the best model and hyperparameters found in the grid search? Find the accuracy of this model on the test set.

In [360]:
# First I will view the best model
bestmodel.best_estimator_.get_params(["classifier"])

{'memory': None,
 'steps': [('minmax', MinMaxScaler()), ('classifier', LogisticRegression())],
 'verbose': False,
 'minmax': MinMaxScaler(),
 'classifier': LogisticRegression(),
 'minmax__clip': False,
 'minmax__copy': True,
 'minmax__feature_range': (0, 1),
 'classifier__C': 1.0,
 'classifier__class_weight': None,
 'classifier__dual': False,
 'classifier__fit_intercept': True,
 'classifier__intercept_scaling': 1,
 'classifier__l1_ratio': None,
 'classifier__max_iter': 100,
 'classifier__multi_class': 'auto',
 'classifier__n_jobs': None,
 'classifier__penalty': 'l2',
 'classifier__random_state': None,
 'classifier__solver': 'lbfgs',
 'classifier__tol': 0.0001,
 'classifier__verbose': 0,
 'classifier__warm_start': False}

As seen above I used the best_estimator_ which allowed me to view the best models learning algorithm and hyperparameters

In [361]:
#Last I will check the accuracy of the model on the test data
bestmodel.score(X_test, y_test)

0.625

## Summarize your results.

As seen above the default KNN classifier model accuracy on the test set for step five had the highest accuracy of 0.6458333333333334. While we saw that the accuracy of the grid search best model on the test set which was 0.6458333333333334 which is the same as above. Then last but not least we saw that the accuracy of the best model on the test set was 0.625 which was a little bit less than that of the default KNN classifier model accuracy on the test set for step five. Over all this was a great exercise that taught me alot. 